In [ ]:
!pip install -U langchain-community
!pip install pypdf
!pip install chromadb
!pip install langchain
!pip install langchain-groq
!pip install sentence_transformers
!pip install numpy
!pip install gradio

In [ ]:
import os
import gradio as gr
from langchain_groq import ChatGroq
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

c:\Users\yesil\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

# .env içindeki GROQ_API_KEY değerini al
groq_api_key = os.getenv("GROQ_API_KEY")

# Modeli başlat
llm = ChatGroq(
    temperature=0,
    groq_api_key=groq_api_key,
    model_name="llama3-70b-8192"
)

# Sorgu gönder
result = llm.invoke("Who is Lord Ram?")
print(result.content)

Lord Ram, also known as Rama, is a legendary king and the seventh avatar (incarnation) of the Hindu god Vishnu. He is a central figure in the Hindu epic, the Ramayana, and is revered as a symbol of virtue, duty, and righteousness.

According to the Ramayana, Lord Ram was born in Ayodhya, the capital of the kingdom of Kosala, to King Dasaratha and Queen Kausalya. He was the eldest of four brothers, including Lakshmana, Bharata, and Shatrughna. Ram was known for his bravery, wisdom, and kindness, and was loved by all in the kingdom.

The story of Lord Ram is a fascinating one. He was exiled to the forest for 14 years by his stepmother, Queen Kaikeyi, who wanted her son Bharata to become the king. During his exile, Ram's wife, Sita, was abducted by the demon king Ravana, who ruled over Lanka. Ram, along with his brother Lakshmana and the monkey god Hanuman, formed an army to rescue Sita and defeat Ravana.

The battle between Ram and Ravana is a legendary one, with Ram ultimately emerging 

In [ ]:
import os
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import gradio as gr

load_dotenv()

def initialize_llm():
    return ChatGroq(
        temperature=0,
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="llama3-70b-8192"
    )

def create_vector_db():
    loader = DirectoryLoader("./data2/", glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory='./chroma_db')
    vector_db.persist()
    return vector_db

# Başlat
llm = initialize_llm()
db_path = "./chroma_db/"
if not os.path.exists(db_path):
    vector_db = create_vector_db()
else:
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

retriever = vector_db.as_retriever()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Prompt'lar
first_prompt = PromptTemplate(
    template="""You are a compassionate cyberbullying guide and mental health chatbot.
Answer the following questions thoughtfully and with the official steps to take.

{context}
User: {question}
Chatbot:""",
    input_variables=["context", "question"]
)

normal_prompt = PromptTemplate(
    template="""
{context}
User: {question}
Chatbot:""",
    input_variables=["context", "question"]
)

# Bu flag ilk mesajda True olacak
first_message = {"used": False}

# Chatbot cevabı
def chatbot_response(user_input, history):
    if not user_input.strip():
        return "Lütfen geçerli bir mesaj girin."

    try:
        prompt = first_prompt if not first_message["used"] else normal_prompt
        first_message["used"] = True  # bir sonraki için artık normal prompt kullan

        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=retriever,
            chain_type_kwargs={"prompt": prompt}
        )

        response = qa_chain.run(user_input)
        return response

    except Exception as e:
        return f"Hata oluştu: {str(e)}"

# Gradio CSS
css = """
.gradio-container {
    background-image: url('data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD...');
    background-size: cover;
    background-position: center;
    background-repeat: no-repeat;
    background-attachment: fixed;
}
"""

# Arayüz
with gr.Blocks(css=css) as app:
    gr.ChatInterface(
        fn=chatbot_response,
        title="Mental Health Chatbot"
    )

app.launch()


c:\Users\yesil\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\yesil\AppData\Local\Temp\ipykernel_7464\2796607472.py:27: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
C:\Users\yesil\AppData\Local\Temp\ipykernel_7464\2796607472.py:29: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are au

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
